#Pet Adoption Prediction


# Data Exploration

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Dense


Using TensorFlow backend.


In [2]:
#link google drive to colab notebook
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
#data directory
data_path = 'gdrive/My Drive/Pet Finder Project/Data/'

#show data files
!ls 'gdrive/My Drive/Pet Finder Project/Data'

all_features.csv  test_metadata.zip   train_metadata_table.csv
breed_labels.csv  test_sentiment      train_metadata.zip
color_labels.csv  test_sentiment.zip  train_sentiment
state_labels.csv  test.zip	      train_sentiment_table.csv
test		  train		      train_sentiment.zip
test_images	  train_images	      train.zip
test_images.zip   train_images.zip
test_metadata	  train_metadata


In [6]:
print('Train')
train = pd.read_csv(data_path + "train/train.csv")
print(train.shape)

print('Breeds')
breeds = pd.read_csv(data_path + "breed_labels.csv")
print(breeds.shape)

print('Colors')
colors = pd.read_csv(data_path + "color_labels.csv")
print(colors.shape)

print('States')
states = pd.read_csv(data_path + "state_labels.csv")
print(states.shape)

print('Metadata')
metadata = pd.read_csv(data_path + "train_metadata_table.csv", sep='\t')
metadata.drop(metadata.columns[0],axis=1,inplace=True) #remove first column it is just indexing
print(metadata.shape)

print('Sentiment')
sentiment = pd.read_csv(data_path + "train_sentiment_table.csv", sep='\t')
sentiment.drop(sentiment.columns[0],axis=1,inplace=True) #remove first column it is just indexing
print(sentiment.shape)

Train
(14993, 24)
Breeds
(307, 3)
Colors
(7, 2)
States
(15, 2)
Metadata
(14993, 13)
Sentiment
(14993, 2)


In [7]:
train.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [8]:
breeds.head()

,BreedID,Type,BreedName
0,1,1,Affenpinscher
1,2,1,Afghan Hound
2,3,1,Airedale Terrier
3,4,1,Akbash
4,5,1,Akita


In [9]:
colors.head()

,ColorID,ColorName
0,1,Black
1,2,Brown
2,3,Golden
3,4,Yellow
4,5,Cream


In [10]:
states.head()

,StateID,StateName
0,41336,Johor
1,41325,Kedah
2,41367,Kelantan
3,41401,Kuala Lumpur
4,41415,Labuan


In [11]:
metadata.head()

,doc_sent_mag,doc_sent_score,vertex_x,vertex_y,bounding_confidence,bounding_importance,dominant_blue,dominant_green,dominant_red,dominant_pixel_frac,dominant_score,label_description,label_score
0,-1,-1,-1,-1,-1.0,-1.0,-1,-1,-1,-1.000000,-1.000000,nothing,-1.000000
1,-1,-1,-1,-1,-1.0,-1.0,-1,-1,-1,-1.000000,-1.000000,nothing,-1.000000
2,-1,-1,299,399,0.8,1.0,51,55,61,0.055064,0.333318,dog,0.960457
3,-1,-1,399,299,0.8,1.0,47,50,54,0.127818,0.136823,dog,0.978698
4,-1,-1,-1,-1,-1.0,-1.0,-1,-1,-1,-1.000000,-1.000000,nothing,-1.000000


In [12]:
metadata['label_description'].value_counts()

nothing                       11440
cat                            1590
dog                            1414
dog breed                       264
dog like mammal                 141
animal shelter                   48
fauna                            31
mammal                           10
dog breed group                   9
black                             4
cage                              4
small to medium sized cats        3
skin                              3
plant                             3
furniture                         3
car                               2
bull terrier                      2
face                              2
nose                              2
blue                              2
yellow                            2
iron                              1
pink                              1
nature reserve                    1
boston terrier                    1
fur                               1
guinea pig                        1
grass                       

In [13]:
sentiment.head()

,doc_sent_mag,doc_sent_score
0,2.4,0.3
1,0.7,-0.2
2,3.7,0.2
3,0.9,0.9
4,3.7,0.6


#Evaluation Metric

In [0]:
#I used code from this github repositery
#https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/quadratic_weighted_kappa.py

import numpy as np


def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Calculates the quadratic weighted kappa
    quadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return 1.0 - numerator / denominator


def linear_weighted_kappa(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Calculates the linear weighted kappa
    linear_weighted_kappa calculates the linear weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    linear_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    linear_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = abs(i - j) / float(num_ratings - 1)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return 1.0 - numerator / denominator


def kappa(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Calculates the kappa
    kappa calculates the kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            if i == j:
                d = 0.0
            else:
                d = 1.0
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return 1.0 - numerator / denominator


def mean_quadratic_weighted_kappa(kappas, weights=None):
    """
    Calculates the mean of the quadratic
    weighted kappas after applying Fisher's r-to-z transform, which is
    approximately a variance-stabilizing transformation.  This
    transformation is undefined if one of the kappas is 1.0, so all kappa
    values are capped in the range (-0.999, 0.999).  The reverse
    transformation is then applied before returning the result.
    mean_quadratic_weighted_kappa(kappas), where kappas is a vector of
    kappa values
    mean_quadratic_weighted_kappa(kappas, weights), where weights is a vector
    of weights that is the same size as kappas.  Weights are applied in the
    z-space
    """
    kappas = np.array(kappas, dtype=float)
    if weights is None:
        weights = np.ones(np.shape(kappas))
    else:
        weights = weights / np.mean(weights)

    # ensure that kappas are in the range [-.999, .999]
    kappas = np.array([min(x, .999) for x in kappas])
    kappas = np.array([max(x, -.999) for x in kappas])

    z = 0.5 * np.log((1 + kappas) / (1 - kappas)) * weights
    z = np.mean(z)
    return (np.exp(2 * z) - 1) / (np.exp(2 * z) + 1)


def weighted_mean_quadratic_weighted_kappa(solution, submission):
    predicted_score = submission[submission.columns[-1]].copy()
    predicted_score.name = "predicted_score"
    if predicted_score.index[0] == 0:
        predicted_score = predicted_score[:len(solution)]
        predicted_score.index = solution.index
    combined = solution.join(predicted_score, how="left")
    groups = combined.groupby(by="essay_set")
    kappas = [quadratic_weighted_kappa(group[1]["essay_score"], group[1]["predicted_score"]) for group in groups]
    weights = [group[1]["essay_weight"].irow(0) for group in groups]
    return mean_quadratic_weighted_kappa(kappas, weights=weights)

In [0]:
#kfold cross validation

def kfold_validation(X_train, y_train, model, epochs=100, num_folds=5, reg=False):

  X_train_folds = np.array_split(X_train, num_folds)
  y_train_folds = np.array_split(y_train, num_folds)
  
  qwk_score = 0
  accuracy  = 0
  
  for i in range(num_folds):
    train_set = np.concatenate((X_train_folds[:i]+X_train_folds[i+1:]))
    label_set = np.concatenate((y_train_folds[:i]+y_train_folds[i+1:]))

    #train
    model.fit(train_set, label_set, epochs=epochs, verbose=0)

    #predict
    y_pred = model.predict_classes(X_train_folds[i])
    
    #if it is regression, round the result to the nearest category
    if(reg == True): 
      y_pred = np.rint(y_pred).astype(int).reshape(-1)
      y_true = y_train_folds[i]
    else:
      y_pred = model.predict_classes(X_train_folds[i])
      y_true = np.argmax(y_train_folds[i].values, axis=1)    
    
   
    #calculate quadratic weighted kappa
    qwk_score += quadratic_weighted_kappa(y_true, y_pred)/num_folds
    
    #calculate accuracy
    accuracy += accuracy_score(y_true, y_pred)/num_folds
    

  print('qwk_score= ', qwk_score, ', accuracy= ', accuracy)
  return [qwk_score, accuracy]

#Benchmark Evaluation

In [0]:
#K cv for benchmark model
#benchmark model that i will use here
#alwyas predict that has adoptability of category 2

def kfold_validation_benchmark(y_train, num_folds=5):

  predicted_category = 2
  
  y_train_folds = np.array_split(y_train, num_folds)
  
  qwk_score = 0
  accuracy  = 0
  
  for i in range(num_folds):
    
    y_true = y_train_folds[i].values
    y_pred = np.full(y_true.shape, predicted_category)

    #calculate quadratic weighted kappa
    qwk_score += quadratic_weighted_kappa(y_true, y_pred)/num_folds
    
    #calculate accuracy
    accuracy += accuracy_score(y_true, y_pred)/num_folds
    

  print('qwk_score= ', qwk_score, ', accuracy= ', accuracy)
  return [qwk_score, accuracy]

In [17]:
kfold_validation_benchmark(train['AdoptionSpeed'])

qwk_score=  0.0 , accuracy=  0.2692592883418333


[0.0, 0.2692592883418333]

#Preprocessing

In [0]:
def handle_missing_data(df, colName, y):
  
  col = df.columns.get_loc(colName)
  
  mean_values = []

  #calculate mean values for each class
  for i in range(5): #5 classes
    mean_values.append(np.mean(df[colName][y == i]))

  #replace missing values with mean value of its class
  for j in range(df.shape[0]):
    if(df.iloc[j,col] == -1):
      df.iloc[j,col] = mean_values[y[j]]

In [0]:
#I will discard these features for now
#['Name', 'Description', 'AdoptionSpeed']

def process_data(data):

  #put processed features to X
  X = pd.DataFrame()

  #convert categorical features to one hot encoding
  categorical_features = ['Type', 'Gender', 'Vaccinated', 'Dewormed', 'RescuerID', 'Sterilized', 'State']

  for f in categorical_features:
    data[f] = pd.Categorical(data[f])
    dfDummies = pd.get_dummies(data[f], prefix = f)
    X = pd.concat([X, dfDummies], axis=1)

  #convert 2 breed feature into one feature
  data['Breed1'] = pd.Categorical(data['Breed1'])
  dfDummies = pd.get_dummies(data['Breed1'], prefix = 'Breed')

  data['Breed2'] = pd.Categorical(data['Breed2'])
  dfDummies = dfDummies + pd.get_dummies(data['Breed2'], prefix = 'Breed')
  X = pd.concat([X, dfDummies], axis=1)

  #convert 3 color feature into one feature
  data['Color1'] = pd.Categorical(data['Color1'])
  dfDummies = pd.get_dummies(data['Color1'], prefix = 'Color')

  data['Color2'] = pd.Categorical(data['Color2'])
  dfDummies = dfDummies + pd.get_dummies(data['Color2'], prefix = 'Color')

  data['Color3'] = pd.Categorical(data['Color3'])
  dfDummies = dfDummies + pd.get_dummies(data['Color3'], prefix = 'Color')

  X = pd.concat([X, dfDummies], axis=1)

  X.fillna(0) 
  
  #remove un repeated values
  for col in X.columns:
    if(X[col].sum() < 40):
      X.drop(col, axis=1, inplace=True)

  #normalize numerical features min-max normalization
  numerical_features = ['Age', 'Fee', 'VideoAmt', 'PhotoAmt', 'MaturitySize', 'FurLength', 'Health', 'Quantity']
  for f in numerical_features:
    normalized_df=(data[f]-data[f].min())/(data[f].max()-data[f].min())
    X = pd.concat([X, normalized_df], axis=1)
    
  return X

In [0]:
def process_sentiment(data, y):

  #put processed features to X
  X = pd.DataFrame()

  #normalize numerical features min-max normalization
  numerical_features = ['doc_sent_mag',	'doc_sent_score']
  for f in numerical_features:
    handle_missing_data(data, f, y)
    normalized_df=(data[f]-data[f].min())/(data[f].max()-data[f].min())
    X = pd.concat([X, normalized_df], axis=1)
    
  return X

In [0]:
def process_metadata(data, y):
  
  #put processed features to X
  X = pd.DataFrame()

  #convert categorical features to one hot encoding
  data['label_description'] = pd.Categorical(data['label_description'])
  dfDummies = pd.get_dummies(data['label_description'], prefix = 'label_description')
  X = pd.concat([X, dfDummies], axis=1)
  
  #remove un repeated values
  for col in X.columns:
    if(X[col].sum() < 11):
      X.drop(col, axis=1, inplace=True)
  
  #normalize numerical features min-max normalization
  for f in data.drop('label_description', axis = 1).columns:
    handle_missing_data(data, f, y)
    normalized_df=(data[f]-data[f].min())/(data[f].max()-data[f].min())
    X = pd.concat([X, normalized_df], axis=1)
    
  return X

#Neural Network

In [0]:
#Classifier model simple network

def cls_model_simple(n_input):
  # create model
  model = Sequential()
  model.add(Dense(12, input_dim=n_input, activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(5, activation='softmax'))

  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
#Regression model simple network

def reg_model_simple(n_input):
	# create model
	model = Sequential()
	model.add(Dense(13, input_dim=n_input, kernel_initializer='normal', activation='relu'))
	model.add(Dense(8, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	
  # Compile model
	model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
	return model

#Implementation

In [0]:
def train_simple_network(features, labels, epochs=100):
  n_input = features.shape[1]
  
  #classifier
  print('Classifier')
  encoded_labels = pd.get_dummies(labels)
  kfold_validation(features, encoded_labels, cls_model_simple(n_input), epochs=epochs)

  #regressor
  print('Regression')
  kfold_validation(features, labels, reg_model_simple(n_input), epochs=epochs, reg=True)

In [25]:
labels = train['AdoptionSpeed']

main_features = process_data(train)
sentiment_features = process_sentiment(sentiment, labels)
metadata_features = process_metadata(metadata, labels)


all_features = pd.concat([main_features, sentiment_features, metadata_features], axis=1)

print(all_features.shape)

(14993, 124)


In [26]:
#train with all features
train_simple_network(all_features, labels, epochs=100)

Classifier
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
qwk_score=  0.0 , accuracy=  0.2799300011277942
Regression
qwk_score=  0.0 , accuracy=  0.2060972292076011


In [34]:
#train with main features
train_simple_network(main_features, labels, epochs=100)

Classifier
qwk_score=  0.3457991777308922 , accuracy=  0.4230656605348325
Regression
qwk_score=  0.0010062659976290077 , accuracy=  0.20549682894075655


#Refinement

In [0]:

important_features =  ['Gender_1', 'Gender_2', 'Gender_3', 'Sterilized_1', 'Sterilized_2', 'Sterilized_3',
'State_41324', 'State_41325', 'State_41326', 'State_41327',
'State_41330', 'State_41332', 'State_41336', 'State_41401', 'Breed_103',
'Breed_109', 'Breed_141', 'Breed_179', 'Breed_189', 'Breed_205',
'Breed_218', 'Breed_254', 'Breed_264', 'Breed_265', 'Breed_266',
'Breed_285', 'Breed_292', 'Breed_299', 'Breed_307', 'Color_3',
'Color_4', 'Color_5', 'Color_6', 'Color_7', 'Age', 'Fee',
'PhotoAmt', 'MaturitySize', 'FurLength',  'Quantity',
'doc_sent_mag',  'vertex_x', 'vertex_y',
'dominant_blue',
'dominant_green', 'dominant_red', 'dominant_pixel_frac',
'dominant_score', 'label_score']

X_important = all_features[important_features]


  

In [32]:
X_important.head()

,Gender_1,Gender_2,Gender_3,Sterilized_1,Sterilized_2,Sterilized_3,State_41324,State_41325,State_41326,State_41327,...,doc_sent_mag,doc_sent_mag,vertex_x,vertex_y,dominant_blue,dominant_green,dominant_red,dominant_pixel_frac,dominant_score,label_score
0,1,0,0,0,1,0,0,0,1,0,...,0.075000,NaN,0.019726,0.027575,0.085691,0.090059,0.108493,0.016975,0.020168,0.031058
1,1,0,0,0,0,1,0,0,0,0,...,0.021875,NaN,0.000000,0.000000,0.084732,0.083124,0.099093,0.000000,0.000000,0.000000
2,1,0,0,0,1,0,0,0,1,0,...,0.115625,NaN,0.389215,0.571085,0.196000,0.205645,0.232000,0.477269,0.647374,0.977815
3,0,1,0,0,1,0,0,0,0,0,...,0.028125,NaN,0.568858,0.392370,0.180000,0.185484,0.204000,0.519992,0.527011,0.989462
4,1,0,0,0,1,0,0,0,1,0,...,0.115625,NaN,0.019726,0.027575,0.085691,0.090059,0.108493,0.016975,0.020168,0.031058


In [33]:
#train with main features
train_simple_network(X_important, labels, epochs=100)

Classifier
qwk_score=  0.0 , accuracy=  0.2799300011277942
Regression
qwk_score=  0.0 , accuracy=  0.2060972292076011
